In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [70]:
train=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
bat=pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')


In [71]:
train.columns

In [72]:
train.describe()

In [73]:
train.info()

In [74]:
train.head()

**Replace the nan values in the 'HomePlanet' with 'other'**

In [75]:
train['HomePlanet']=train['HomePlanet'].apply(lambda x:str(x))
def batata(x):
    if (x=='nan'):
        return 'other'
    else :
        return x

train['HomePlanet']=train['HomePlanet'].map(lambda y:batata(y))


In [76]:

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['HomePlanet']=le.fit_transform(train['HomePlanet'])

**In the 'CryoSleep' column , i gave the 'True' value 1 and the 'False' value 0 and the null value 2**

In [77]:

train['CryoSleep']=train['CryoSleep'].apply(lambda x:str(x))
def cryo(x):
    if x=='False':
        return 0
    if x=='True':
        return 1
    else:
        return 2
train['CryoSleep']=train['CryoSleep'].apply(lambda x:cryo(x))


**In the "Cabin" column, i replaced the 'nan' values with 'T/0/P'(found it online) then applied the 'mapp' function**

In [78]:

train['Cabin'].fillna('T/0/P',inplace=True)
train[['n1','n2','n3']]=train['Cabin'].str.split('/',expand=True)
train.drop('Cabin',axis=1,inplace=True)


In [79]:

mapping = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "H": 8, "I": 9, "J": 10, "K": 11, "L": 12, "M": 13, "N": 14, "O": 15, "P": 16, "Q":17,"R":18, "S": 19, "T": 20,"U": 21, "V": 22, "W": 23, "X": 24, "Y": 25, "Z":26}

def mapp(string, mapping):
    return int(''.join([str(mapping.get(x, x)) for x in string]))
train['n1']=train['n1'].apply(lambda x:mapp(x,mapping))
train['n3']=train['n3'].apply(lambda x:mapp(x,mapping))


**Destination columns**

In [80]:

train['Destination']=train['Destination'].apply(lambda x:str(x))
train['Destination'].replace({'nan':'other'},inplace=True)

from sklearn.preprocessing import OneHotEncoder
ohe2=OneHotEncoder(categories='auto',  drop=None, sparse=True, handle_unknown='ignore' )      
transformed = ohe2.fit_transform(train[['Destination']])
x=pd.DataFrame(transformed.toarray())
x.columns=ohe2.categories_
x.index=train.index
what=[]
for y in x.columns:
    what.append(y[0])
train[what]=x
train.drop('Destination',axis=1,inplace=True)

**'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Age'**

In [81]:

gg=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Age']
for q in gg:
    train[q]=train[q].fillna(value=np.mean(train[q]))

**VIP**

In [82]:

train['VIP'].fillna(3,inplace=True)
train['VIP'].replace({False:0,True:1},inplace=True)


In [83]:
train.drop('Name',axis=1,inplace=True)
train.set_index("PassengerId",inplace=True)

**Transported**

In [84]:
train['Transported']=train['Transported'].apply(lambda x:str(x))
train['Transported']=train['Transported'].apply(lambda x:cryo(x))

**Now i'll apply the same preprocessing to the test set**

In [85]:
test['HomePlanet']=test['HomePlanet'].apply(lambda x:str(x))
def batata(x):
    if (x=='nan'):
        return 'other'
    else :
        return x

test['HomePlanet']=test['HomePlanet'].map(lambda y:batata(y))

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
test['HomePlanet']=le.fit_transform(test['HomePlanet'])
test['CryoSleep']=test['CryoSleep'].apply(lambda x:str(x))
def cryo(x):
    if x=='False':
        return 0
    if x=='True':
        return 1
    else:
        return 2
test['CryoSleep']=test['CryoSleep'].apply(lambda x:cryo(x))

test['Cabin'].fillna('T/0/P',inplace=True)
test[['n1','n2','n3']]=test['Cabin'].str.split('/',expand=True)
test.drop('Cabin',axis=1,inplace=True)

mapping = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "H": 8, "I": 9, "J": 10, "K": 11, "L": 12, "M": 13, "N": 14, "O": 15, "P": 16, "Q":17,"R":18, "S": 19, "T": 20,"U": 21, "V": 22, "W": 23, "X": 24, "Y": 25, "Z":26}

def mapp(string, mapping):
    return int(''.join([str(mapping.get(x, x)) for x in string]))
test['n1']=test['n1'].apply(lambda x:mapp(x,mapping))
test['n3']=test['n3'].apply(lambda x:mapp(x,mapping))

test['Destination']=test['Destination'].apply(lambda x:str(x))
test['Destination'].replace({'nan':'other'},inplace=True)

from sklearn.preprocessing import OneHotEncoder
ohe2=OneHotEncoder(categories='auto',  drop=None, sparse=True, handle_unknown='ignore' )      
transformed = ohe2.fit_transform(test[['Destination']])
x=pd.DataFrame(transformed.toarray())
x.columns=ohe2.categories_
x.index=test.index
what=[]
for y in x.columns:
    what.append(y[0])
test[what]=x
test.drop('Destination',axis=1,inplace=True)
gg=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Age']
for q in gg:
    test[q]=test[q].fillna(value=np.mean(test[q]))
test['VIP'].fillna(3,inplace=True)
test['VIP'].replace({False:0,True:1},inplace=True)

test.set_index("PassengerId",inplace=True)

s

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(train.drop('Transported',axis=1),train['Transported'], test_size=0.33, random_state=42)

In [87]:
from sklearn.neural_network import MLPClassifier

#clf= make_pipeline(StandardScaler(), MLPClassifier(random_state=1, max_iter=300,learning_rate='adaptive',hidden_layer_sizes=(100,10)).fit(X_train, y_train))
#clf.fit(X_train,y_train)
#clf.score(X_dev,y_dev)

In [88]:
a=MLPClassifier(random_state=1,hidden_layer_sizes=(100,10))

parameters = {"learning_rate_init":[0.001,0.01,0.1],
              "max_iter":[200,300],
              "learning_rate":['invscaling','adaptive'],
              "solver":['sgd','adam'],
              "activation":['tanh','relu'],
              "momentum":[0.9,0.99,0.998],
              "beta_1":[0.9,0.99,0.998],
              "beta_2":[0.999,0.998,0.99]
             }

In [89]:
#from sklearn.model_selection import GridSearchCV
#grid = GridSearchCV(estimator=a, param_grid = parameters, cv = 2, n_jobs=-1,verbose=2,)
#grid.fit(X_dev, y_dev)
#print(" Results from Grid Search " )
#print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
#print("\n The best score across ALL searched params:\n",grid.best_score_)
#print("\n The best parameters across ALL searched params:\n",grid.best_params_)

In [90]:
model=MLPClassifier(activation='tanh', beta_2=0.99, hidden_layer_sizes=(100, 10),learning_rate='invscaling', learning_rate_init=0.01,)

In [91]:
model.fit(X_train,y_train)
model.score(X_train,y_train)

In [92]:
model.score(X_dev,y_dev)

In [93]:
predictions=model.predict(test.drop('Name',axis=1))
t=pd.DataFrame(predictions,index=test.index,columns=['Transported'])

In [94]:
#t.to_csv('Daher.csv')

In [100]:
output = pd.DataFrame({'PassengerId': test.index, 'Transported': predictions})
output['Transported'] = output['Transported'].astype('bool')
output.to_csv('submission.csv', index=False)

In [99]:
output